<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/DL/9_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D0%B0_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Архитектура модели трансформеров

## Введение

Современные задачи обработки естественного языка (NLP) требуют мощных моделей, способных эффективно обрабатывать большие объемы текстовых данных. Одним из наиболее значимых прорывов в этой области стало появление архитектуры **трансформеров** (Transformers). Эти модели стали основой для таких известных моделей, как BERT, GPT и T5, которые показали выдающиеся результаты на различных задачах NLP.

В этой лекции мы подробно рассмотрим основные компоненты архитектуры трансформеров, их функциональность и особенности. Мы также поговорим о том, почему именно эта архитектура стала настолько популярной и успешной.



## 1. История возникновения трансформеров

До появления трансформеров, основными архитектурами для обработки последовательностей были рекуррентные нейронные сети (RNN), долгосрочная краткосрочная память (LSTM) и гейтед рекуррентные сети (GRU). Однако эти модели имели свои ограничения:

- **Зависимость от порядка**: RNN обрабатывают данные последовательно, что делает их неэффективными при длинных последовательностях.
- **Проблема с долгосрочной зависимостью**: LSTM и GRU решают проблему долгосрочной зависимости лучше, но все равно страдают от затруднений при обучении на длинных последовательностях.
- **Параллельизация**: Поскольку RNN работают последовательно, параллельное выполнение операций становится невозможным, что замедляет обучение.

Трансформеры, представленные в статье "Attention is All You Need" в 2017 году, предложили новый подход к обработке последовательностей, который устраняет многие из этих недостатков.



## 2. Основные принципы работы трансформера



### 2.1. Модуль внимания (Attention Mechanism)

Механизм внимания является ключевым компонентом архитектуры трансформеров и позволяет модели эффективно учитывать зависимости между различными частями входных данных. В данном разделе мы подробно рассмотрим, как работает механизм внимания, что означают матрицы $Q$, $K$, $V$, параметр $d_k$ и функция активации $\text{softmax}$.

#### Пример для понимания

Предположим, у нас есть два предложения:
1. "Я люблю читать книги."
2. "Мне нравится играть в футбол."

Когда модель должна перевести это предложение на другой язык, она может использовать механизм внимания, чтобы "уделять внимание" различным словам в зависимости от контекста. Например, если модель работает над переводом слова "читать", она будет уделять больше внимания слову "книги", чем другим словам в предложении. Это потому, что слово "читать" имеет сильную семантическую связь с "книгами".

Теперь давайте разберем формально, как это реализуется в механизме внимания.

#### Формула механизма внимания

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Где:
- $Q$ — матрица запросов (queries),
- $K$ — матрица ключей (keys),
- $V$ — матрица значений (values),
- $d_k$ — размерность ключей,
- $\text{softmax}$ — функция активации softmax, которая используется для нормализации весов.

#### Разбор каждого элемента:

##### 1. Матрица запросов ($Q$):

Матрица запросов $Q$ представляет собой набор векторов запросов. Каждый вектор запроса соответствует определенному элементу входной последовательности, который модель пытается "понять" или "перевести". Например, если модель работает с предложением "Я люблю читать книги.", то каждый из слов "Я", "люблю", "читать", "книги" будет иметь свой вектор запроса в матрице $Q$.

Формально, если у нас есть последовательность длины $T$ и размерность скрытого пространства $d_{model}$, то $Q \in \mathbb{R}^{T \times d_{model}}$.

##### 2. Матрица ключей ($K$):

Матрица ключей $K$ также представляет собой набор векторов ключей, которые используются для вычисления степени совпадения между запросами и элементами входной последовательности. Эти ключи помогают модели определить, какие элементы входной последовательности наиболее релевантны для текущего запроса.

Формально, $K \in \mathbb{R}^{T \times d_{model}}$.

##### 3. Матрица значений ($V$):

Матрица значений $V$ содержит информацию, которую модель будет использовать для генерации выходного представления. После того как модель определила степень релевантности каждого элемента входной последовательности (используя матрицы $Q$ и $K$), она использует значения из матрицы $V$ для создания окончательного выходного представления.

Формально, $V \in \mathbb{R}^{T \times d_{model}}$.

##### 4. Размерность ключей ($d_k$):

Размерность ключей $d_k$ — это размерность векторов ключей и запросов. Обычно $d_k$ равен размерности скрытого пространства $d_{model}$. Однако в многослойном внимании (multi-head attention) $d_k$ может быть меньше, чтобы уменьшить количество параметров и улучшить производительность модели.

##### 5. Функция активации softmax:

Функция активации softmax применяется к результату скалярного произведения $QK^T$ для нормализации весов. Она преобразует эти веса в вероятности, которые показывают, насколько каждый элемент входной последовательности важен для текущего запроса.

Формально, если у нас есть матрица $S \in \mathbb{R}^{T \times T}$, где $S_{ij}$ — это скалярное произведение $q_i \cdot k_j$, то softmax применяется по строкам этой матрицы:

$$
\text{softmax}(S)_{ij} = \frac{\exp(S_{ij})}{\sum_{k=1}^T \exp(S_{ik})}
$$

Это означает, что сумма всех весов в каждой строке будет равна 1, что делает их интерпретируемыми как вероятности.

#### Как работает механизм внимания:

1. **Вычисление скалярного произведения**: Для каждого запроса $q_i$ вычисляется скалярное произведение с каждым ключом $k_j$. Это дает нам матрицу $S \in \mathbb{R}^{T \times T}$, где каждый элемент $S_{ij}$ показывает степень совпадения между запросом $i$ и ключом $j$.

2. **Нормализация весов**: Скалярное произведение $S$ нормализуется с помощью функции softmax, чтобы получить матрицу весов $A \in \mathbb{R}^{T \times T}$. Эта матрица показывает, насколько каждый элемент входной последовательности важен для текущего запроса.

3. **Вычисление контекстного вектора**: Матрица весов $A$ умножается на матрицу значений $V$, чтобы получить контекстный вектор для каждого запроса. Этот контекстный вектор представляет собой комбинацию значений входной последовательности, взвешенных по их степени релевантности для текущего запроса.

Формально, контекстный вектор $C$ вычисляется следующим образом:

$$
C = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Этот контекстный вектор затем используется для генерации выходного представления модели.




### Конкретный числовой пример механизма внимания

Давайте рассмотрим конкретный числовой пример, чтобы лучше понять, как работает механизм внимания. Предположим, у нас есть простая последовательность из трех слов: "Я", "люблю", "книги". Мы будем использовать однослойное внимание для демонстрации.

#### Шаг 1: Входные данные и проекция

Представим каждое слово в виде вектора размерности $d_{model} = 4$. Для простоты возьмем следующие случайные векторы:

- "Я" -> $x_1 = [0.1, 0.2, 0.3, 0.4]$
- "люблю" -> $x_2 = [0.5, 0.6, 0.7, 0.8]$
- "книги" -> $x_3 = [0.9, 1.0, 1.1, 1.2]$

Матрица входных данных $X$ будет выглядеть так:

$$
X = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
$$

Теперь мы используем обучаемые параметры для получения матриц запросов $Q$, ключей $K$ и значений $V$:

$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$

где $W_Q, W_K, W_V \in \mathbb{R}^{d_{model} \times d_k}$.

Для простоты предположим, что $d_k = 4$ (то есть размерность скрытого пространства равна размерности ключей). Возьмем случайные значения для $W_Q, W_K, W_V$:

$$
W_Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}, \quad
W_K = \begin{bmatrix}
0.2 & 0.3 & 0.4 & 0.5 \\
0.6 & 0.7 & 0.8 & 0.9 \\
1.0 & 1.1 & 1.2 & 1.3 \\
1.4 & 1.5 & 1.6 & 1.7
\end{bmatrix}, \quad
W_V = \begin{bmatrix}
0.3 & 0.4 & 0.5 & 0.6 \\
0.7 & 0.8 & 0.9 & 1.0 \\
1.1 & 1.2 & 1.3 & 1.4 \\
1.5 & 1.6 & 1.7 & 1.8
\end{bmatrix}
$$

Теперь вычислим матрицы $Q$, $K$ и $V$:

$$
Q = XW_Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
= \begin{bmatrix}
1.4 & 1.6 & 1.8 & 2.0 \\
3.4 & 3.8 & 4.2 & 4.6 \\
5.4 & 6.0 & 6.6 & 7.2
\end{bmatrix}
$$

$$
K = XW_K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 & 0.4 & 0.5 \\
0.6 & 0.7 & 0.8 & 0.9 \\
1.0 & 1.1 & 1.2 & 1.3 \\
1.4 & 1.5 & 1.6 & 1.7
\end{bmatrix}
= \begin{bmatrix}
1.8 & 2.0 & 2.2 & 2.4 \\
4.2 & 4.6 & 5.0 & 5.4 \\
6.6 & 7.2 & 7.8 & 8.4
\end{bmatrix}
$$

$$
V = XW_V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 & 0.5 & 0.6 \\
0.7 & 0.8 & 0.9 & 1.0 \\
1.1 & 1.2 & 1.3 & 1.4 \\
1.5 & 1.6 & 1.7 & 1.8
\end{bmatrix}
= \begin{bmatrix}
2.2 & 2.4 & 2.6 & 2.8 \\
5.0 & 5.4 & 5.8 & 6.2 \\
7.8 & 8.4 & 9.0 & 9.6
\end{bmatrix}
$$

#### Шаг 2: Вычисление скалярного произведения

Вычислим скалярное произведение $QK^T$:

$$
QK^T = \begin{bmatrix}
1.4 & 1.6 & 1.8 & 2.0 \\
3.4 & 3.8 & 4.2 & 4.6 \\
5.4 & 6.0 & 6.6 & 7.2
\end{bmatrix}
\begin{bmatrix}
1.8 & 4.2 & 6.6 \\
2.0 & 4.6 & 7.2 \\
2.2 & 5.0 & 7.8 \\
2.4 & 5.4 & 8.4
\end{bmatrix}
= \begin{bmatrix}
14.8 & 34.4 & 54.0 \\
34.4 & 81.2 & 128.0 \\
54.0 & 128.0 & 202.0
\end{bmatrix}
$$

#### Шаг 3: Нормализация весов с помощью softmax

Применим функцию softmax к каждой строке матрицы $QK^T / \sqrt{d_k}$:

$$
\text{softmax}(S)_{ij} = \frac{\exp(S_{ij})}{\sum_{k=1}^T \exp(S_{ik})}
$$

Для $d_k = 4$:

$$
\frac{QK^T}{\sqrt{d_k}} = \frac{1}{\sqrt{4}} \begin{bmatrix}
14.8 & 34.4 & 54.0 \\
34.4 & 81.2 & 128.0 \\
54.0 & 128.0 & 202.0
\end{bmatrix}
= \begin{bmatrix}
7.4 & 17.2 & 27.0 \\
17.2 & 40.6 & 64.0 \\
27.0 & 64.0 & 101.0
\end{bmatrix}
$$

Теперь применим softmax:

$$
\text{softmax}\left(\begin{bmatrix}
7.4 & 17.2 & 27.0 \\
17.2 & 40.6 & 64.0 \\
27.0 & 64.0 & 101.0
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.0000 & 0.0001 & 0.9999 \\
0.0001 & 0.0047 & 0.9952 \\
0.0002 & 0.0133 & 0.9865
\end{bmatrix}
$$

#### Шаг 4: Вычисление контекстного вектора

Умножим матрицу весов на матрицу значений $V$:

$$
C = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
\approx \begin{bmatrix}
0.0000 & 0.0001 & 0.9999 \\
0.0001 & 0.0047 & 0.9952 \\
0.0002 & 0.0133 & 0.9865
\end{bmatrix}
\begin{bmatrix}
2.2 & 2.4 & 2.6 & 2.8 \\
5.0 & 5.4 & 5.8 & 6.2 \\
7.8 & 8.4 & 9.0 & 9.6
\end{bmatrix}
\approx \begin{bmatrix}
7.8 & 8.4 & 9.0 & 9.6 \\
7.8 & 8.4 & 9.0 & 9.6 \\
7.8 & 8.4 & 9.0 & 9.6
\end{bmatrix}
$$

Таким образом, контекстный вектор для каждого слова в последовательности будет приближенно равен $[7.8, 8.4, 9.0, 9.6]$.







### 2.2. Масштабируемое точечное произведение (Scaled Dot-Product Attention)

Масштабируемое точечное произведение внимания (Scaled Dot-Product Attention) — это конкретный тип механизма внимания, который используется в архитектуре трансформеров. Этот механизм позволяет модели эффективно учитывать зависимости между различными элементами входной последовательности и масштабирует значения, чтобы избежать слишком больших или маленьких значений во время обучения.

#### Принцип работы

Основная идея заключается в вычислении скалярного произведения между матрицей запросов $Q$ и матрицей ключей $K$, что позволяет определить степень совпадения между запросами и ключами. Затем результат этого скалярного произведения нормализуется с помощью функции softmax для получения весов, которые показывают, насколько каждый элемент входной последовательности важен для текущего запроса. Наконец, эти веса используются для взвешивания значений из матрицы $V$, чтобы получить контекстный вектор.

Формула масштабируемого точечного произведения внимания:

$$
\text{Scaled Dot-Product Attention} = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Где:
- $Q$ — матрица запросов,
- $K$ — матрица ключей,
- $V$ — матрица значений,
- $d_k$ — размерность ключей,
- $\text{softmax}$ — функция активации softmax, которая используется для нормализации весов.

#### Подробное объяснение каждого компонента:

1. **Скалярное произведение $QK^T$**:
   - Скалярное произведение между матрицей запросов $Q$ и матрицей ключей $K$ дает нам матрицу, где каждый элемент $S_{ij}$ показывает степень совпадения между запросом $i$ и ключом $j$.
   - Это произведение может привести к большим значениям, особенно если размерность $d_k$ велика. Чтобы избежать проблем с градиентами при обучении, результат скалярного произведения делится на $\sqrt{d_k}$.

2. **Масштабирование $\frac{QK^T}{\sqrt{d_k}}$**:
   - Масштабирование помогает стабилизировать процесс обучения, предотвращая слишком большие или маленькие значения в выходных данных.
   - Например, если размерность $d_k$ равна 64, то результат скалярного произведения будет делиться на $\sqrt{64} = 8$.

3. **Нормализация с помощью softmax**:
   - Функция softmax применяется к каждой строке результата скалярного произведения, чтобы преобразовать его в вероятности.
   - Это означает, что сумма всех весов в каждой строке будет равна 1, что делает их интерпретируемыми как вероятности.
   - Формально, если у нас есть матрица $S \in \mathbb{R}^{T \times T}$, где $S_{ij}$ — это скалярное произведение $q_i \cdot k_j$, то softmax применяется по строкам этой матрицы:

  $$
     \text{softmax}(S)_{ij} = \frac{\exp(S_{ij})}{\sum_{k=1}^T \exp(S_{ik})}
  $$

4. **Взвешивание значений $V$**:
   - После нормализации весов с помощью softmax, они умножаются на матрицу значений $V$ для получения контекстного вектора.
   - Каждый элемент контекстного вектора представляет собой комбинацию значений входной последовательности, взвешенных по их степени релевантности для текущего запроса.

Таким образом, масштабируемое точечное произведение внимания позволяет модели эффективно учитывать зависимости между различными элементами входной последовательности и создавать точные выходные представления.



### Конкретный числовой пример для масштабируемого точечного произведения внимания (Scaled Dot-Product Attention)

Давайте рассмотрим конкретный числовой пример, чтобы лучше понять, как работает масштабируемое точечное произведение внимания. Предположим, у нас есть простая последовательность из трех слов: "Я", "люблю", "книги". Мы будем использовать однослойное внимание для демонстрации.

#### Шаг 1: Входные данные и проекция

Представим каждое слово в виде вектора размерности $d_{model} = 4$. Для простоты возьмем следующие случайные векторы:

- "Я" -> $x_1 = [0.1, 0.2, 0.3, 0.4]$
- "люблю" -> $x_2 = [0.5, 0.6, 0.7, 0.8]$
- "книги" -> $x_3 = [0.9, 1.0, 1.1, 1.2]$

Матрица входных данных $X$ будет выглядеть так:

$$
X = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
$$

Теперь мы используем обучаемые параметры для получения матриц запросов $Q$, ключей $K$ и значений $V$:

$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$

где $W_Q, W_K, W_V \in \mathbb{R}^{d_{model} \times d_k}$.

Для простоты предположим, что $d_k = 2$ (размерность скрытого пространства меньше размерности модели). Возьмем случайные значения для $W_Q, W_K, W_V$:

$$
W_Q = \begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}, \quad
W_K = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_V = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
$$

Теперь вычислим матрицы $Q$, $K$ и $V$:

$$
Q = XW_Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}
= \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
$$

$$
K = XW_K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
$$

$$
V = XW_V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
$$

#### Шаг 2: Вычисление скалярного произведения

Вычислим скалярное произведение $QK^T$:

$$
QK^T = \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
\begin{bmatrix}
0.6 & 1.4 & 2.2 \\
0.7 & 1.5 & 2.3
\end{bmatrix}
= \begin{bmatrix}
0.82 & 1.84 & 2.86 \\
1.62 & 3.64 & 5.66 \\
2.42 & 5.44 & 8.46
\end{bmatrix}
$$

#### Шаг 3: Масштабирование

Размерность ключей $d_k = 2$, поэтому масштабируем результат на $\sqrt{d_k} = \sqrt{2} \approx 1.414$:

$$
\frac{QK^T}{\sqrt{d_k}} = \frac{1}{\sqrt{2}} \begin{bmatrix}
0.82 & 1.84 & 2.86 \\
1.62 & 3.64 & 5.66 \\
2.42 & 5.44 & 8.46
\end{bmatrix}
\approx \begin{bmatrix}
0.58 & 1.30 & 2.02 \\
1.15 & 2.57 & 4.00 \\
1.71 & 3.85 & 6.00
\end{bmatrix}
$$

#### Шаг 4: Нормализация весов с помощью softmax

Применим функцию softmax к каждой строке матрицы $\frac{QK^T}{\sqrt{d_k}}$:

$$
\text{softmax}(S)_{ij} = \frac{\exp(S_{ij})}{\sum_{k=1}^T \exp(S_{ik})}
$$

Для первой строки:

$$
\text{softmax}\left(\begin{bmatrix}
0.58 & 1.30 & 2.02
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67
\end{bmatrix}
$$

Для второй строки:

$$
\text{softmax}\left(\begin{bmatrix}
1.15 & 2.57 & 4.00
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.03 & 0.11 & 0.86
\end{bmatrix}
$$

Для третьей строки:

$$
\text{softmax}\left(\begin{bmatrix}
1.71 & 3.85 & 6.00
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.01 & 0.05 & 0.94
\end{bmatrix}
$$

Таким образом, нормализованная матрица весов:

$$
\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) \approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
$$

#### Шаг 5: Взвешивание значений $V$

Умножим матрицу весов на матрицу значений $V$:

$$
C = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
\begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
\approx \begin{bmatrix}
1.84 & 1.92 \\
2.07 & 2.16 \\
2.21 & 2.30
\end{bmatrix}
$$


Этот числовой пример показывает, как механизм масштабируемого точечного произведения внимания позволяет модели фокусироваться на наиболее релевантных частях входной последовательности. В данном случае модель уделяет больше внимания последнему слову "книги" для всех элементов последовательности, что может быть полезно в задачах перевода или анализа текста.









### 2.3. Многослойное внимание (Multi-Head Attention)

Многослойное внимание (multi-head attention) — это расширение механизма внимания, которое позволяет модели улучшить представление информации за счет использования нескольких параллельных механизмов внимания. Каждый из этих механизмов фокусируется на разных аспектах данных, что позволяет модели улавливать более сложные зависимости и взаимосвязи между элементами входной последовательности.

#### Принцип работы

Основная идея многослойного внимания заключается в том, что вместо одного механизма внимания используются несколько "голов" (heads). Каждая голова выполняет собственный механизм внимания с отдельными параметрами, а затем результаты всех голов объединяются в одно общее представление. Это позволяет модели лучше учитывать различные аспекты входной последовательности и создавать более точные выходные представления.

Формула многослойного внимания:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(head_1, head_2, ..., head_h)W^O
$$

Где:
- $head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$,
- $W_i^Q$, $W_i^K$, $W_i^V$ — параметры для каждого головного механизма,
- $W^O$ — проекционная матрица для объединения всех головных механизмов.

#### Подробное объяснение каждого компонента:

1. **Отдельные головы (Heads)**:
   - Для каждого головного механизма используются отдельные параметры $W_i^Q$, $W_i^K$, $W_i^V$, которые применяются к исходным матрицам запросов $Q$, ключей $K$ и значений $V$.
   - Эти параметры позволяют каждой голове выделять разные аспекты данных. Например, одна голова может фокусироваться на семантических связях между словами, а другая — на грамматических структурах.
   - Формально, для каждой головы $i$:

  $$
     Q_i = QW_i^Q, \quad K_i = KW_i^K, \quad V_i = VW_i^V
  $$

     Где $W_i^Q \in \mathbb{R}^{d_{model} \times d_k}$, $W_i^K \in \mathbb{R}^{d_{model} \times d_k}$, $W_i^V \in \mathbb{R}^{d_{model} \times d_v}$.

   - Здесь $d_k$ и $d_v$ — размерности ключей и значений соответственно. Обычно $d_k = d_v$, но они могут быть различными.

2. **Механизм внимания для каждой головы**:
   - После применения параметров $W_i^Q$, $W_i^K$, $W_i^V$, каждая голова выполняет механизм внимания на своих преобразованных матрицах $Q_i$, $K_i$, $V_i$.
   - Формально, для каждой головы $i$:

  $$
     head_i = \text{Attention}(Q_i, K_i, V_i)
  $$

   - Механизм внимания для каждой головы может быть масштабируемым точечным произведением внимания (scaled dot-product attention), который мы рассмотрели ранее:

  $$
     \text{Attention}(Q_i, K_i, V_i) = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_k}}\right)V_i
  $$

   - Таким образом, каждая голова вычисляет свой контекстный вектор, основываясь на различных аспектах входной последовательности.

3. **Объединение результатов всех голов**:
   - После того как все головы выполнили свои механизмы внимания, их результаты объединяются в одно общее представление.
   - Это достигается путем конкатенации всех контекстных векторов $head_i$ по оси последней размерности:

  $$
     \text{Concat}(head_1, head_2, ..., head_h)
  $$

   - Результат конкатенации имеет форму $T \times h \cdot d_v$, где $T$ — длина последовательности, $h$ — количество голов, $d_v$ — размерность значений.

4. **Проекционная матрица $W^O$**:
   - После конкатенации результатов всех голов используется проекционная матрица $W^O$, чтобы привести объединенное представление обратно к исходной размерности $d_{model}$.
   - Формально, проекционная матрица $W^O \in \mathbb{R}^{h \cdot d_v \times d_{model}}$.
   - Умножение результата конкатенации на $W^O$ дает окончательное выходное представление:

  $$
     \text{MultiHead}(Q, K, V) = \text{Concat}(head_1, head_2, ..., head_h)W^O
  $$

   - Этот шаг позволяет модели сохранить согласованность размерностей и интегрировать информацию из всех голов в единое представление.

#### Преимущества многослойного внимания:

1. **Улучшенное представление информации**:
   - Использование нескольких голов позволяет модели улавливать различные аспекты данных, что делает ее более мощной и гибкой.
   - Например, одна голова может фокусироваться на семантических связях, другая — на грамматической структуре, третья — на лексических особенностях и т.д.

2. **Параллелизация**:
   - Поскольку каждая голова работает независимо, механизмы внимания можно выполнять параллельно, что значительно ускоряет обучение и инференс (выполнение модели на новых данных).

3. **Снижение вероятности переобучения**:
   - Использование нескольких голов увеличивает количество параметров модели, что помогает избежать переобучения на малых наборах данных.

4. **Гибкость**:
   - Многослойное внимание можно адаптировать для различных задач NLP, таких как машинный перевод, классификация текста, суммаризация текста и другие.


Таким образом, многослойное внимание является важным компонентом архитектуры трансформеров и позволяет модели эффективно улавливать сложные зависимости и взаимосвязи между элементами входной последовательности. Благодаря использованию нескольких параллельных механизмов внимания, каждый из которых фокусируется на разных аспектах данных, модель становится более мощной и гибкой, что делает ее подходящей для решения широкого спектра задач обработки естественного языка.







### Конкретный числовой пример многослойного внимания (Multi-Head Attention)

Давайте рассмотрим конкретный числовой пример для многослойного внимания. Предположим, у нас есть простая последовательность из трех слов: "Я", "люблю", "книги". Мы будем использовать двухголовое внимание (two-head attention) для демонстрации.

#### Шаг 1: Входные данные и проекция

Представим каждое слово в виде вектора размерности $d_{model} = 4$. Для простоты возьмем следующие случайные векторы:

- "Я" -> $x_1 = [0.1, 0.2, 0.3, 0.4]$
- "люблю" -> $x_2 = [0.5, 0.6, 0.7, 0.8]$
- "книги" -> $x_3 = [0.9, 1.0, 1.1, 1.2]$

Матрица входных данных $X$ будет выглядеть так:

$$
X = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
$$

Теперь мы используем обучаемые параметры для получения матриц запросов $Q$, ключей $K$ и значений $V$:

$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$

где $W_Q, W_K, W_V \in \mathbb{R}^{d_{model} \times d_k}$.

Для простоты предположим, что $d_k = 2$ (размерность скрытого пространства меньше размерности модели). Возьмем случайные значения для параметров голов:

**Голова 1:**

$$
W_1^Q = \begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}, \quad
W_1^K = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_1^V = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
$$

**Голова 2:**

$$
W_2^Q = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_2^K = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}, \quad
W_2^V = \begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
$$

Теперь вычислим матрицы $Q_i$, $K_i$ и $V_i$ для каждой головы:

**Голова 1:**

$$
Q_1 = XW_1^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}
= \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
$$

$$
K_1 = XW_1^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
$$

$$
V_1 = XW_1^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2 = XW_2^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
$$

$$
K_2 = XW_2^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
$$

$$
V_2 = XW_2^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
= \begin{bmatrix}
0.8 & 0.9 \\
1.6 & 1.7 \\
2.4 & 2.5
\end{bmatrix}
$$

#### Шаг 2: Вычисление скалярного произведения для каждой головы

**Голова 1:**

$$
Q_1K_1^T = \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
\begin{bmatrix}
0.6 & 1.4 & 2.2 \\
0.7 & 1.5 & 2.3
\end{bmatrix}
= \begin{bmatrix}
0.82 & 1.84 & 2.86 \\
1.62 & 3.64 & 5.66 \\
2.42 & 5.44 & 8.46
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2K_2^T = \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
\begin{bmatrix}
0.7 & 1.5 & 2.3 \\
0.8 & 1.6 & 2.4
\end{bmatrix}
= \begin{bmatrix}
1.06 & 2.34 & 3.62 \\
2.18 & 4.80 & 7.42 \\
3.30 & 7.26 & 11.22
\end{bmatrix}
$$

#### Шаг 3: Масштабирование

Размерность ключей $d_k = 2$, поэтому масштабируем результат на $\sqrt{d_k} = \sqrt{2} \approx 1.414$:

**Голова 1:**

$$
\frac{Q_1K_1^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.58 & 1.30 & 2.02 \\
1.15 & 2.57 & 4.00 \\
1.71 & 3.85 & 6.00
\end{bmatrix}
$$

**Голова 2:**

$$
\frac{Q_2K_2^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.75 & 1.65 & 2.56 \\
1.54 & 3.40 & 5.25 \\
2.33 & 5.13 & 7.93
\end{bmatrix}
$$

#### Шаг 4: Нормализация весов с помощью softmax

Применим функцию softmax к каждой строке матрицы $\frac{Q_iK_i^T}{\sqrt{d_k}}$:

**Голова 1:**

$$
\text{softmax}\left(\begin{bmatrix}
0.58 & 1.30 & 2.02 \\
1.15 & 2.57 & 4.00 \\
1.71 & 3.85 & 6.00
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
$$

**Голова 2:**

$$
\text{softmax}\left(\begin{bmatrix}
0.75 & 1.65 & 2.56 \\
1.54 & 3.40 & 5.25 \\
2.33 & 5.13 & 7.93
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.07 & 0.22 & 0.71 \\
0.03 & 0.10 & 0.87 \\
0.02 & 0.07 & 0.91
\end{bmatrix}
$$

#### Шаг 5: Взвешивание значений $V_i$

Умножим матрицу весов на матрицу значений $V_i$:

**Голова 1:**

$$
C_1 = \text{softmax}\left(\frac{Q_1K_1^T}{\sqrt{d_k}}\right)V_1
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
\begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
\approx \begin{bmatrix}
1.84 & 1.92 \\
2.07 & 2.16 \\
2.21 & 2.30
\end{bmatrix}
$$

**Голова 2:**

$$
C_2 = \text{softmax}\left(\frac{Q_2K_2^T}{\sqrt{d_k}}\right)V_2
\approx \begin{bmatrix}
0.07 & 0.22 & 0.71 \\
0.03 & 0.10 & 0.87 \\
0.02 & 0.07 & 0.91
\end{bmatrix}
\begin{bmatrix}
0.8 & 0.9 \\
1.6 & 1.7 \\
2.4 & 2.5
\end{bmatrix}
\approx \begin{bmatrix}
1.90 & 1.99 \\
2.13 & 2.24 \\
2.27 & 2.37
\end{bmatrix}
$$

#### Шаг 6: Объединение результатов всех голов

Конкатенация результатов всех голов по оси последней размерности:

$$
\text{Concat}(C_1, C_2) = \begin{bmatrix}
1.84 & 1.92 & 1.90 & 1.99 \\
2.07 & 2.16 & 2.13 & 2.24 \\
2.21 & 2.30 & 2.27 & 2.37
\end{bmatrix}
$$

#### Шаг 7: Проекционная матрица $W^O$

Используем проекционную матрицу $W^O$ для приведения объединенного представления обратно к исходной размерности $d_{model}$. Предположим, что:

$$
W^O = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
$$

Выполним умножение:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(C_1, C_2)W^O
\approx \begin{bmatrix}
1.84 & 1.92 & 1.90 & 1.99 \\
2.07 & 2.16 & 2.13 & 2.24 \\
2.21 & 2.30 & 2.27 & 2.37
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
\approx \begin{bmatrix}
4.66 & 4.86 & 5.06 & 5.26 \\
5.26 & 5.46 & 5.66 & 5.86 \\
5.86 & 6.06 & 6.26 & 6.46
\end{bmatrix}
$$

Таким образом, этот числовой пример показывает, как механизм многослойного внимания позволяет модели фокусироваться на различных аспектах входной последовательности. Каждая голова выделяет разные зависимости между элементами последовательности, а затем их результаты объединяются в одно общее представление. Это делает модель более мощной и гибкой, что позволяет ей эффективно решать различные задачи обработки естественного языка.




## 3. Структура трансформера

Трансформер — это архитектура нейронной сети, которая используется для обработки последовательностей данных, таких как тексты. Основные компоненты трансформера включают **энкодер** (encoder) и **декодер** (decoder). В зависимости от задачи, может использоваться только энкодер или оба блока вместе.

### 3.1. Энкодер

Энкодер является первой частью трансформера и предназначен для преобразования входной последовательности в более абстрактное представление. Каждый слой энкодера состоит из нескольких ключевых компонентов, которые обеспечивают эффективное обучение и обработку данных:

#### Многослойное внимание (Multi-Head Attention)

Многослойное внимание позволяет модели фокусироваться на различных частях входной последовательности. Этот механизм основан на параллельном выполнении нескольких голов внимания, каждая из которых выделяет разные аспекты данных.

Каждая голова выполняет следующие шаги:
1. Применяет обучаемые параметры $W_i^Q$, $W_i^K$, $W_i^V$ к исходным матрицам запросов $Q$, ключей $K$ и значений $V$.
2. Выполняет механизм внимания для каждой головы с помощью масштабируемого точечного произведения внимания (scaled dot-product attention).
3. Объединяет результаты всех голов с помощью конкатенации и проекционной матрицы $W^O$.

Формально, многослойное внимание можно записать как:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(head_1, head_2, ..., head_h)W^O
$$

Где:
- $head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$,
- $W_i^Q$, $W_i^K$, $W_i^V$ — параметры для каждого головного механизма,
- $W^O$ — проекционная матрица для объединения всех головных механизмов.

#### Передача сигнала через сеть (Feed-Forward Neural Network)

После применения многослойного внимания выход передается через полноценную нейронную сеть (feed-forward neural network). Это позволяет добавить нелинейность в модель и улучшить ее способность к обучению сложных зависимостей.

Слои нейронной сети обычно состоят из двух линейных преобразований с функцией активации ReLU между ними:

$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

Где:
- $W_1 \in \mathbb{R}^{d_{model} \times d_{ff}}$ и $W_2 \in \mathbb{R}^{d_{ff} \times d_{model}}$ — весовые матрицы,
- $b_1 \in \mathbb{R}^{d_{ff}}$ и $b_2 \in \mathbb{R}^{d_{model}}$ — векторы смещений,
- $d_{ff}$ — размерность скрытого слоя нейронной сети (обычно значительно больше, чем $d_{model}$).

#### Слои нормализации (Layer Normalization) и остаточные соединения (Residual Connections)

Чтобы стабилизировать процесс обучения и улучшить производительность модели, между компонентами энкодера применяются слои нормализации и остаточные соединения.

**Слой нормализации (Layer Normalization)**:
- Нормализует значения по оси последней размерности, чтобы сделать их менее чувствительными к изменению входных данных.
- Формально, нормализация выполняется по формуле:

$$
\text{LayerNorm}(x) = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta
$$

Где:
- $\mu$ и $\sigma^2$ — среднее и дисперсия значений по оси последней размерности,
- $\gamma$ и $\beta$ — обучаемые параметры,
- $\epsilon$ — малое значение для предотвращения деления на ноль.

**Остаточные соединения (Residual Connections)**:
- Добавляют входные данные к выходу компонента, чтобы сохранить информацию о начальных данных и улучшить градиентный спуск.
- Формально, остаточное соединение выполняется по формуле:

$$
\text{Residual}(x) = x + f(x)
$$

Где $f(x)$ — это выход компонента (например, многослойного внимания или нейронной сети).




### 3.2. Декодер


Декодер является важной частью архитектуры трансформеров (например, в модели Transformer), которая используется для генерации последовательности на основе входного представления, созданного энкодером. Декодер имеет несколько ключевых компонентов и механизмов, которые позволяют модели учитывать как входные данные от энкодера, так и уже сгенерированные слова при создании следующего слова.

#### 1. Многослойное внимание на декодере

**Многослойное внимание на декодере** работает аналогично многослойному вниманию в энкодере, но с некоторыми отличиями. В декодере этот механизм фокусируется на различных частях уже сгенерированной части текста. Это означает, что модель может "смотреть" на предыдущие слова в последовательности, чтобы определить, какое слово должно быть сгенерировано следующим.

- **Запросы $Q$**: В этом контексте запросы представляют собой текущее состояние декодера.
- **Ключи $K$ и значения $V$**: Эти параметры берутся из того же слоя декодера, что и запросы, поскольку механизм внимания работает внутри одной последовательности.

Формально, механизм можно записать как:
$$
\text{SelfAttention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$
где $d_k$ — размерность ключей.

#### 2. Механизм внимания на декодере (Decoder Attention)

**Механизм внимания на декодере** позволяет модели учитывать не только уже сгенерированные слова, но и входные данные от энкодера. Этот механизм использует два набора данных: выход энкодера и уже сгенерированные слова.

- **Запросы $Q$**: Запросы берутся из текущего слоя декодера.
- **Ключи $K$ и значения $V$**: Эти параметры берутся из выхода энкодера.

Формально, механизм можно записать как:
$$
\text{DecoderAttention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Здесь важно отметить, что механизмы внимания на декодере могут работать как с маской, так и без неё. При обучении модели с использованием автоперевода или других задач, где порядок слов важен, часто применяется маска, чтобы модель не могла "подглядывать" в будущее. Это делается для того, чтобы модель училась предсказывать следующее слово, основываясь только на уже виденных словах.

#### 3. Передача сигнала через сеть (Feed-Forward Neural Network)

**Передача сигнала через сеть** добавляет нелинейность в модель и работает аналогично нейронной сети в энкодере. После применения механизма внимания сигнал проходит через полностью связную нейронную сеть, состоящую из двух линейных слоёв и функции активации ReLU между ними.

Формально, это можно записать как:
$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$
где $W_1$, $W_2$ — весовые матрицы, а $b_1$, $b_2$ — векторы смещений.

#### Слои нормализации и остаточные соединения

Как и в энкодере, между компонентами декодера также применяются слои нормализации и остаточные соединения. Эти элементы помогают стабилизировать процесс обучения и улучшить производительность модели.

- **Нормализация**: Нормализация применяется к каждому слою после вычисления внимания и передачи сигнала через сеть. Она помогает стабилизировать градиенты и ускоряет процесс обучения.
- **Остаточные соединения**: Остаточные соединения позволяют пропускать прямой сигнал через несколько слоёв, что помогает решить проблему затухания градиентов и улучшает обобщение модели.



 ### Конкретный числовой пример структуры трансформера

Давайте рассмотрим конкретный числовой пример для работы энкодера и декодера в трансформере. Предположим, у нас есть простая последовательность из трех слов: "Я", "люблю", "книги". Мы будем использовать однослойный энкодер и однослойный декодер для демонстрации.

#### Шаг 1: Входные данные и проекция

Представим каждое слово в виде вектора размерности $d_{model} = 4$. Для простоты возьмем следующие случайные векторы:

- "Я" -> $x_1 = [0.1, 0.2, 0.3, 0.4]$
- "люблю" -> $x_2 = [0.5, 0.6, 0.7, 0.8]$
- "книги" -> $x_3 = [0.9, 1.0, 1.1, 1.2]$

Матрица входных данных $X$ будет выглядеть так:

$$
X = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
$$

#### Шаг 2: Энкодер

##### Многослойное внимание (Multi-Head Attention)

Предположим, что мы используем двухголовое внимание (two-head attention). Возьмем случайные значения для параметров голов:

**Голова 1:**

$$
W_1^Q = \begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}, \quad
W_1^K = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_1^V = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
$$

**Голова 2:**

$$
W_2^Q = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_2^K = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}, \quad
W_2^V = \begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
$$

Вычислим матрицы $Q_i$, $K_i$ и $V_i$ для каждой головы:

**Голова 1:**

$$
Q_1 = XW_1^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}
= \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
$$

$$
K_1 = XW_1^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
$$

$$
V_1 = XW_1^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2 = XW_2^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
$$

$$
K_2 = XW_2^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
$$

$$
V_2 = XW_2^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2
\end{bmatrix}
\begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
= \begin{bmatrix}
0.8 & 0.9 \\
1.6 & 1.7 \\
2.4 & 2.5
\end{bmatrix}
$$

##### Вычисление скалярного произведения для каждой головы

**Голова 1:**

$$
Q_1K_1^T = \begin{bmatrix}
0.5 & 0.6 \\
1.1 & 1.2 \\
1.7 & 1.8
\end{bmatrix}
\begin{bmatrix}
0.6 & 1.4 & 2.2 \\
0.7 & 1.5 & 2.3
\end{bmatrix}
= \begin{bmatrix}
0.82 & 1.84 & 2.86 \\
1.62 & 3.64 & 5.66 \\
2.42 & 5.44 & 8.46
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2K_2^T = \begin{bmatrix}
0.6 & 0.7 \\
1.4 & 1.5 \\
2.2 & 2.3
\end{bmatrix}
\begin{bmatrix}
0.7 & 1.5 & 2.3 \\
0.8 & 1.6 & 2.4
\end{bmatrix}
= \begin{bmatrix}
1.06 & 2.34 & 3.62 \\
2.18 & 4.80 & 7.42 \\
3.30 & 7.26 & 11.22
\end{bmatrix}
$$

##### Масштабирование и нормализация весов с помощью softmax

Размерность ключей $d_k = 2$, поэтому масштабируем результат на $\sqrt{d_k} = \sqrt{2} \approx 1.414$:

**Голова 1:**

$$
\frac{Q_1K_1^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.58 & 1.30 & 2.02 \\
1.15 & 2.57 & 4.00 \\
1.71 & 3.85 & 6.00
\end{bmatrix}
$$

Применим функцию softmax к каждой строке матрицы $\frac{Q_1K_1^T}{\sqrt{d_k}}$:

$$
\text{softmax}\left(\begin{bmatrix}
0.58 & 1.30 & 2.02 \\
1.15 & 2.57 & 4.00 \\
1.71 & 3.85 & 6.00
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
$$

**Голова 2:**

$$
\frac{Q_2K_2^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.75 & 1.65 & 2.56 \\
1.54 & 3.40 & 5.25 \\
2.33 & 5.13 & 7.93
\end{bmatrix}
$$

Применим функцию softmax к каждой строке матрицы $\frac{Q_2K_2^T}{\sqrt{d_k}}$:

$$
\text{softmax}\left(\begin{bmatrix}
0.75 & 1.65 & 2.56 \\
1.54 & 3.40 & 5.25 \\
2.33 & 5.13 & 7.93
\end{bmatrix}\right)
\approx \begin{bmatrix}
0.07 & 0.22 & 0.71 \\
0.03 & 0.10 & 0.87 \\
0.02 & 0.07 & 0.91
\end{bmatrix}
$$

##### Взвешивание значений $V_i$

**Голова 1:**

$$
C_1 = \text{softmax}\left(\frac{Q_1K_1^T}{\sqrt{d_k}}\right)V_1
\approx \begin{bmatrix}
0.09 & 0.24 & 0.67 \\
0.03 & 0.11 & 0.86 \\
0.01 & 0.05 & 0.94
\end{bmatrix}
\begin{bmatrix}
0.7 & 0.8 \\
1.5 & 1.6 \\
2.3 & 2.4
\end{bmatrix}
\approx \begin{bmatrix}
1.84 & 1.92 \\
2.07 & 2.16 \\
2.21 & 2.30
\end{bmatrix}
$$

**Голова 2:**

$$
C_2 = \text{softmax}\left(\frac{Q_2K_2^T}{\sqrt{d_k}}\right)V_2
\approx \begin{bmatrix}
0.07 & 0.22 & 0.71 \\
0.03 & 0.10 & 0.87 \\
0.02 & 0.07 & 0.91
\end{bmatrix}
\begin{bmatrix}
0.8 & 0.9 \\
1.6 & 1.7 \\
2.4 & 2.5
\end{bmatrix}
\approx \begin{bmatrix}
1.90 & 1.99 \\
2.13 & 2.24 \\
2.27 & 2.37
\end{bmatrix}
$$

##### Объединение результатов всех голов

Конкатенация результатов всех голов по оси последней размерности:

$$
\text{Concat}(C_1, C_2) = \begin{bmatrix}
1.84 & 1.92 & 1.90 & 1.99 \\
2.07 & 2.16 & 2.13 & 2.24 \\
2.21 & 2.30 & 2.27 & 2.37
\end{bmatrix}
$$

##### Проекционная матрица $W^O$

Используем проекционную матрицу $W^O$ для приведения объединенного представления обратно к исходной размерности $d_{model}$. Предположим, что:

$$
W^O = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
$$

Выполним умножение:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(C_1, C_2)W^O
\approx \begin{bmatrix}
1.84 & 1.92 & 1.90 & 1.99 \\
2.07 & 2.16 & 2.13 & 2.24 \\
2.21 & 2.30 & 2.27 & 2.37
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
\approx \begin{bmatrix}
4.66 & 4.86 & 5.06 & 5.26 \\
5.26 & 5.46 & 5.66 & 5.86 \\
5.86 & 6.06 & 6.26 & 6.46
\end{bmatrix}
$$

##### Передача сигнала через сеть (Feed-Forward Neural Network)

Добавим нелинейную трансформацию с помощью нейронной сети:

$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

Предположим, что:

$$
W_1 = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}, \quad
W_2 = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
$$

$$
b_1 = \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}, \quad
b_2 = \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}
$$

Выполним вычисления:

$$
\text{FFN}(x) = \max(0, \begin{bmatrix}
4.66 & 4.86 & 5.06 & 5.26 \\
5.26 & 5.46 & 5.66 & 5.86 \\
5.86 & 6.06 & 6.26 & 6.46
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
+ \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix})
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
+ \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}
$$




После того как энкодер преобразовал входную последовательность в абстрактное представление, декодер использует это представление для генерации новой последовательности слов. Декодер имеет схожую структуру с энкодером, но содержит дополнительные компоненты, которые позволяют модели учитывать уже сгенерированную часть текста при создании следующего слова.

#### Шаг 3: Декодер

##### Входные данные для декодера

Допустим, что мы хотим перевести фразу "Я люблю книги" на другой язык (например, английский). Предположим, что первое слово, которое модель должна сгенерировать, — это "I". Мы будем использовать этот вектор для входных данных декодера:

- "I" -> $y_1 = [0.1, 0.2, 0.3, 0.4]$

Матрица входных данных для декодера $Y$ будет выглядеть так:

$$
Y = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
$$

##### Многослойное внимание на декодере

Как и в энкодере, декодер использует многослойное внимание для обработки входной последовательности. Для простоты возьмем те же параметры голов, что и для энкодера:

**Голова 1:**

$$
W_1^Q = \begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}, \quad
W_1^K = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_1^V = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
$$

**Голова 2:**

$$
W_2^Q = \begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}, \quad
W_2^K = \begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}, \quad
W_2^V = \begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
$$

Вычислим матрицы $Q_i$, $K_i$ и $V_i$ для каждой головы:

**Голова 1:**

$$
Q_1 = YW_1^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 \\
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8
\end{bmatrix}
= \begin{bmatrix}
0.5 & 0.6
\end{bmatrix}
$$

$$
K_1 = YW_1^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7
\end{bmatrix}
$$

$$
V_1 = YW_1^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2 = YW_2^Q = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.2 & 0.3 \\
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.6 & 0.7
\end{bmatrix}
$$

$$
K_2 = YW_2^K = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.3 & 0.4 \\
0.5 & 0.6 \\
0.7 & 0.8 \\
0.9 & 1.0
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8
\end{bmatrix}
$$

$$
V_2 = YW_2^V = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4
\end{bmatrix}
\begin{bmatrix}
0.4 & 0.5 \\
0.6 & 0.7 \\
0.8 & 0.9 \\
1.0 & 1.1
\end{bmatrix}
= \begin{bmatrix}
0.8 & 0.9
\end{bmatrix}
$$

##### Вычисление скалярного произведения для каждой головы

**Голова 1:**

$$
Q_1K_1^T = \begin{bmatrix}
0.5 & 0.6
\end{bmatrix}
\begin{bmatrix}
0.6 \\
0.7
\end{bmatrix}
= \begin{bmatrix}
0.82
\end{bmatrix}
$$

**Голова 2:**

$$
Q_2K_2^T = \begin{bmatrix}
0.6 & 0.7
\end{bmatrix}
\begin{bmatrix}
0.7 \\
0.8
\end{bmatrix}
= \begin{bmatrix}
1.06
\end{bmatrix}
$$

##### Масштабирование и нормализация весов с помощью softmax

Размерность ключей $d_k = 2$, поэтому масштабируем результат на $\sqrt{d_k} = \sqrt{2} \approx 1.414$:

**Голова 1:**

$$
\frac{Q_1K_1^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.58
\end{bmatrix}
$$

Применим функцию softmax к каждой строке матрицы $\frac{Q_1K_1^T}{\sqrt{d_k}}$:

$$
\text{softmax}\left(\begin{bmatrix}
0.58
\end{bmatrix}\right)
\approx \begin{bmatrix}
1.0
\end{bmatrix}
$$

**Голова 2:**

$$
\frac{Q_2K_2^T}{\sqrt{d_k}} \approx \begin{bmatrix}
0.75
\end{bmatrix}
$$

Применим функцию softmax к каждой строке матрицы $\frac{Q_2K_2^T}{\sqrt{d_k}}$:

$$
\text{softmax}\left(\begin{bmatrix}
0.75
\end{bmatrix}\right)
\approx \begin{bmatrix}
1.0
\end{bmatrix}
$$

##### Взвешивание значений $V_i$

**Голова 1:**

$$
C_1 = \text{softmax}\left(\frac{Q_1K_1^T}{\sqrt{d_k}}\right)V_1
\approx \begin{bmatrix}
1.0
\end{bmatrix}
\begin{bmatrix}
0.7 & 0.8
\end{bmatrix}
= \begin{bmatrix}
0.7 & 0.8
\end{bmatrix}
$$

**Голова 2:**

$$
C_2 = \text{softmax}\left(\frac{Q_2K_2^T}{\sqrt{d_k}}\right)V_2
\approx \begin{bmatrix}
1.0
\end{bmatrix}
\begin{bmatrix}
0.8 & 0.9
\end{bmatrix}
= \begin{bmatrix}
0.8 & 0.9
\end{bmatrix}
$$

##### Объединение результатов всех голов

Конкатенация результатов всех голов по оси последней размерности:

$$
\text{Concat}(C_1, C_2) = \begin{bmatrix}
0.7 & 0.8 & 0.8 & 0.9
\end{bmatrix}
$$

##### Проекционная матрица $W^O$

Используем проекционную матрицу $W^O$ для приведения объединенного представления обратно к исходной размерности $d_{model}$. Возьмем ту же проекционную матрицу, что и для энкодера:

$$
W^O = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
$$

Выполним умножение:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(C_1, C_2)W^O
\approx \begin{bmatrix}
0.7 & 0.8 & 0.8 & 0.9
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
\approx \begin{bmatrix}
1.78 & 1.90 & 2.02 & 2.14
\end{bmatrix}
$$

##### Передача сигнала через сеть (Feed-Forward Neural Network)

Добавим нелинейную трансформацию с помощью нейронной сети:

$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

Предположим, что:

$$
W_1 = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}, \quad
W_2 = \begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
$$

$$
b_1 = \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}, \quad
b_2 = \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}
$$

Выполним вычисления:

$$
\text{FFN}(x) = \max(0, \begin{bmatrix}
1.78 & 1.90 & 2.02 & 2.14
\end{bmatrix}
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
+ \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix})
\begin{bmatrix}
0.1 & 0.2 & 0.3 & 0.4 \\
0.5 & 0.6 & 0.7 & 0.8 \\
0.9 & 1.0 & 1.1 & 1.2 \\
1.3 & 1.4 & 1.5 & 1.6
\end{bmatrix}
+ \begin{bmatrix}
0.1 \\
0.2 \\
0.3 \\
0.4
\end{bmatrix}
$$

Таким образом, этот числовой пример показывает, как работает полный цикл трансформера, включая энкодер и декодер. Энкодер преобразует входную последовательность в абстрактное представление, а декодер использует это представление для генерации новой последовательности текста. Использование многослойного внимания, нейронной сети и различных техник нормализации и остаточных соединений позволяет модели эффективно решать задачи обработки естественного языка, такие как машинный перевод.






## 4. Преимущества трансформеров

Трансформеры (Transformers) — это архитектура нейронных сетей, которая была введена в 2017 году и быстро стала доминирующей моделью для задач обработки естественного языка (NLP). В этом разделе мы подробно рассмотрим основные преимущества трансформеров по сравнению с другими моделями, такими как рекуррентные нейронные сети (RNN).

### 4.1. Параллелизация

#### Описание механизма
Одним из ключевых преимуществ трансформеров является их способность к параллельной обработке данных. В отличие от RNN, которые должны последовательно обрабатывать элементы входной последовательности, трансформеры могут обрабатывать все элементы одновременно. Это возможно благодаря тому, что трансформеры не имеют рекуррентных связей, которые ограничивают возможность параллельного выполнения операций.

В RNN каждая ячейка зависит от предыдущей, что делает невозможным обработку нескольких элементов одновременно. Например, чтобы вычислить выходное значение на шаге t, необходимо знать выходное значение на шаге t-1. Это приводит к значительным времененным затратам при обучении и инференсе, особенно на длинных последовательностях.

Трансформеры же используют механизм внимания (attention mechanism), который позволяет модели сразу учитывать все элементы последовательности без необходимости последовательной обработки. Таким образом, все элементы могут быть обработаны параллельно, что значительно ускоряет процесс обучения и инференса.

#### Пример
Представьте себе задачу машинного перевода, где входная последовательность состоит из 100 слов. В случае RNN каждое слово будет обрабатываться последовательно, что может занять значительное время, особенно если модель имеет много слоев. В случае трансформера все 100 слов будут обработаны одновременно, что значительно сокращает время обработки.

#### Результаты
Эта способность к параллельной обработке данных делает трансформеры гораздо более эффективными на больших объемах данных. В экспериментах было показано, что трансформеры могут обучаться в несколько раз быстрее, чем RNN, что делает их идеальными для задач, требующих обработки больших текстовых корпусов.

### 4.2. Устойчивость к длинным последовательностям

#### Описание механизма
Другим важным преимуществом трансформеров является их способность эффективно работать с длинными последовательностями. Механизмы внимания позволяют модели учитывать связи между всеми элементами последовательности, даже если они находятся далеко друг от друга. Это делает трансформеры более устойчивыми к длинным последовательностям, чем RNN.

В RNN проблема заключается в том, что информация теряется при прохождении через большое количество временных шагов. Это связано с эффектом затухания градиентов (vanishing gradient problem), когда градиенты становятся слишком маленькими, чтобы эффективно обновлять веса модели. Это приводит к тому, что RNN плохо справляются с длинными последовательностями.

Трансформеры решают эту проблему за счет использования механизма внимания. Механизм внимания позволяет модели "смотреть" на все элементы последовательности одновременно и определять, какие из них наиболее важны для текущего контекста. Это позволяет модели учитывать долгосрочные зависимости между элементами последовательности, что делает ее более устойчивой к длинным последовательностям.

#### Пример
Рассмотрим задачу анализа текста, где входная последовательность состоит из нескольких тысяч слов. В случае RNN информация о первых словах будет потеряна к моменту, когда модель дойдет до конца последовательности. В случае трансформера модель сможет эффективно учитывать все слова, даже те, которые находятся на большом расстоянии друг от друга.

#### Результаты
Это свойство делает трансформеры идеальными для задач, где важна долгосрочная зависимость, таких как машинный перевод, анализ длинных текстов и другие задачи NLP.

### 4.3. Гибкость

#### Описание механизма
Трансформеры обладают высокой гибкостью и могут быть адаптированы для решения различных задач NLP. Благодаря своей универсальной архитектуре, трансформеры можно использовать для таких задач, как машинный перевод, классификация текста, суммаризация текста, вопросно-ответные системы и многие другие.

Основная причина такой гибкости заключается в том, что трансформеры не зависят от специфической структуры данных, как это часто бывает в других моделях. Они могут работать с различными типами текстовых данных, будь то предложения, абзацы или даже целые документы.

Кроме того, трансформеры могут быть легко адаптированы для работы с дополнительными данными, такими как изображения или звуковые сигналы. Это делает их универсальным инструментом для разработчиков, работающих в различных областях, таких как компьютерное зрение, анализ временных рядов и другие.

#### Пример
Представьте себе задачу классификации текста, где нужно определить, относится ли данный текст к определенной категории. Трансформеры могут быть легко адаптированы для этой задачи, просто добавив выходной слой, который будет выполнять классификацию. Аналогично, трансформеры могут быть использованы для задачи машинного перевода, просто заменив выходной слой на слой декодирования.

#### Результаты
Эта гибкость делает трансформеры одним из самых популярных инструментов в области машинного обучения. Они широко используются в различных задачах, начиная от простых задач классификации и заканчивая сложными задачами, такими как создание искусственного интеллекта для игры в шахматы или Go.


## 5. Заключение

Архитектура трансформеров представляет собой важный шаг вперед в развитии моделей обработки естественного языка. Благодаря своей способности эффективно обрабатывать длинные последовательности и возможности параллельного выполнения операций, трансформеры достигли выдающихся результатов на множестве задач NLP.

На данный момент трансформеры являются одним из самых перспективных направлений в области машинного обучения, и их использование продолжает расширяться в различных областях, таких как компьютерное зрение, анализ временных рядов и другие. Исследователи активно работают над улучшением трансформеров, разрабатывая новые архитектуры и методы обучения, что позволяет им решать все более сложные задачи.



## Вопросы для обсуждения:

### 1. Какие проблемы решаются с помощью механизма внимания?

Механизм внимания решает проблему зависимости между элементами последовательности, особенно на больших расстояниях. В рекуррентных нейронных сетях (RNN) информация может теряться при прохождении через большое количество временных шагов, что приводит к проблеме затухания градиентов. Механизм внимания позволяет модели учитывать связи между всеми элементами последовательности, даже если они находятся далеко друг от друга. Это особенно полезно в задачах, где важна долгосрочная зависимость, такие как машинный перевод, анализ длинных текстов и другие.

### 2. Почему трансформеры более эффективны, чем RNN на длинных последовательностях?

Трансформеры более эффективны на длинных последовательностях благодаря двум основным факторам: параллелизации и механизму внимания. Во-первых, трансформеры могут обрабатывать все элементы последовательности одновременно, что значительно ускоряет процесс обучения и инференса. Во-вторых, механизм внимания позволяет модели учитывать долгосрочные зависимости между элементами последовательности, что делает их более устойчивыми к длинным последовательностям. В RNN эти зависимости теряются из-за эффекта затухания градиентов, что делает их менее эффективными на длинных последовательностях.

### 3. Какие задачи NLP можно решить с помощью трансформеров?

Трансформеры могут быть использованы для решения широкого спектра задач NLP, включая:
- **Машинный перевод**: Трансформеры могут эффективно переводить тексты с одного языка на другой, учитывая долгосрочные зависимости между словами.
- **Классификация текста**: Трансформеры могут быть использованы для классификации текстов на различные категории, такие как позитивные/негативные отзывы, новости по темам и другие.
- **Суммаризация текста**: Трансформеры могут создавать краткие резюме длинных текстов, сохраняя ключевые моменты.
- **Вопросно-ответные системы**: Трансформеры могут быть использованы для создания систем, которые отвечают на вопросы пользователя на основе предоставленных данных.
- **Извлечение информации**: Трансформеры могут извлекать важную информацию из текста, такую как имена людей, организации, даты и другие.



